# Markings Dataset
The purpose of this notebook is to quickly create datasets for reporting purposes

## Imports
The packages used for this are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [arcgis](https://esri.github.io/arcgis-python-api/apidoc/html/) to search for markings feature layer dataset

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd

## Constants

In [8]:
gis = GIS("https://austin.maps.arcgis.com/home/index.html")
url = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/TRANSPORTATION_{}/FeatureServer/0"
col = ['CREW_ASSIGNED','TOTALS']

## Create Feature Layers and DataFrames
This will create feature layers and convert to dataframe. This will take a while since there are over 30000 rows

In [23]:
sdf = pd.DataFrame.spatial.from_layer(FeatureLayer(url.format("markings_specialty_point")))
year = pd.DataFrame({'YEARS':[4,4,6,6],col[0]:['CBD','BIKE','SIGNAL','OTHER']})

In [4]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import SpatialDataFrame

# setup feature layer to update
# client_id='CrnxPfTcm7Y7ZGl7'
gis = GIS("https://austin.maps.arcgis.com/home/index.html")

url = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/{}/FeatureServer/1"
feature_name = "Data_Tracker_Signs_Markings"
out_path = r"G:\ATD\Signs_and_Markings\MARKINGS\Markings WORK ORDERS"
shp_name = "markings_work_orders.shp"

fl = FeatureLayer(url.format(feature_name))

sdf = pd.DataFrame.spatial.from_layer(fl)
sdf.to_featureclass(out_location=out_path, out_name=shp_name)

AttributeError: 'DataFrame' object has no attribute 'to_featureclass'

## Create Tables

In [24]:
sp_sdf = sdf.copy().rename(columns={'SHAPE':'TOTALS'})
sp_sdf = sp_sdf.groupby([col[0]]).count()[[col[1]]].reset_index().filter(items=col)

final = pd.merge(sp_sdf,year,on=[col[0]])
final['ANNUAL_TOTALS'] = round(final.TOTALS / final.YEARS)
final = final[[final.columns[0],final.columns[2],final.columns[1],final.columns[3]]]

The table will be displayed below

In [25]:
display(final)

,CREW_ASSIGNED,YEARS,TOTALS,ANNUAL_TOTALS
0,BIKE,4,9590,2398.0
1,CBD,4,2367,592.0
2,OTHER,6,13077,2180.0
3,SIGNAL,6,8326,1388.0


In [26]:
final.to_csv(r'C:\Users\Govs\Projects\Files\specialty_markings_totals.csv')